<a href="https://colab.research.google.com/github/DanielDialektico/dialektico-machine-learning-practices/blob/main/notebooks/Otros/Tutorial_API_de_Spotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://dialektico.com/wp-content/uploads/2023/03/MiniLogoW4.png" alt="Dialéktico Logo" />

# **Consumo de API de Spotify con Python usando Spotipy 🎼**

# **Introducción**

En este **notebook** pondremos en práctica cómo consumir la **API** de **Spotify** usando la librería Spotipy de Python, donde obtendremos datos de canciones y artistas, y los ordenaremos en formato tabular para finalmente almacenar la información en un CSV.

Este notebook es parte del [tutorial de cómo consumir la API de Spotify con Python](https://dialektico.com/tutorial-api-spotify-python/), donde se pueden hallar instrucciones más detalladas.

<br>
<center><img src="https://dialektico.com/wp-content/uploads/2025/01/APIS_colab.jpg" width="300" /></center>

<br>
<center><img src="https://dialektico.com/wp-content/uploads/2025/01/APIS_Colab_obj.jpg" width="300" /></center>

<br>

# **Instalación de librerías**

Instalamos la librerías:

***Nota*:** *Las instalaciones se realizan a pesar de que algunas librerías ya están integradas de forma nativa en Colab, esto para asegurar que el Notebook no presente problemas de ejecución si se dan cambios de sintaxis entre versiones de librerías.*

In [1]:
!pip install pandas==2.2.2
!pip install tqdm==4.67.1
!pip install spotipy==2.25.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 6.6 MB/s eta 0:00:00


<br>

# **Se añaden las credenciales de autenticación**

Se declaran las variables con las **credenciales** ([ver tutorial](https://dialektico.com/tutorial-api-spotify-python/)).

In [2]:
# Se importan las librerías necesarias.
import warnings
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from tqdm import tqdm

# Se filtran las advertencias.
warnings.filterwarnings('ignore')

# Se declara el ID del cliente.
client_id = '1ac2cc06cf774a9caa548b37a4e22ddf'

# Se declara el valor del secret client.
client_secret = '401d357f92584b80b26f40f9f8664eaa'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)

<br>

# **Usando Spotipy para extraer datos**

Comenzamos creando el objeto que nos permitirá consumir los endpoints de la API.

In [3]:
# Se crea el cliente que se utilizará para consumir la API.
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Probamos utilizando la función search, la cual permite obtener información del catálogo de Spotify sobre **álbumes**, **artistas**,** listas de reproducción**, **canciones**, **programas**, **episodios** o **audiolibros** que coincidan con una cadena de palabras clave (es decir, buscará y traerá información sobre algún elemento especificado).

Visualizamos el formato en el que se entrega la información, haciendo una búsqueda de datos de la agrupación **Blackpink**.

In [4]:
# Se busca información sobre Blackpink.
result = sp.search(q='Blackpink', type='artist', limit=1)

# Se imprime el resultado.
result

{'artists': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=1&query=Blackpink&type=artist',
  'limit': 1,
  'next': 'https://api.spotify.com/v1/search?offset=1&limit=1&query=Blackpink&type=artist',
  'offset': 0,
  'previous': None,
  'total': 833,
  'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/41MozSoPIsD1dJM0CLPjZF'},
    'followers': {'href': None, 'total': 52784037},
    'genres': ['k-pop'],
    'href': 'https://api.spotify.com/v1/artists/41MozSoPIsD1dJM0CLPjZF',
    'id': '41MozSoPIsD1dJM0CLPjZF',
    'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5ebc9690bc711d04b3d4fd4b87c',
      'height': 640,
      'width': 640},
     {'url': 'https://i.scdn.co/image/ab67616100005174c9690bc711d04b3d4fd4b87c',
      'height': 320,
      'width': 320},
     {'url': 'https://i.scdn.co/image/ab6761610000f178c9690bc711d04b3d4fd4b87c',
      'height': 160,
      'width': 160}],
    'name': 'BLACKPINK',
    'popularity': 78,
    'type': 'artist',
    

Ahora accedemos a un elemento en particular del diccionario, en este caso, al **número total de seguidores** del grupo en la plataforma:

In [5]:
# Se imprime el número de seguidores de Blackpink.
result['artists']['items'][0]['followers']['total']

52784037

Obtenemos el top de 10 canciones más populares de la banda utilizando el método [artist_top_tracks](https://spotipy.readthedocs.io/en/2.22.1/#spotipy.client.Spotify.artist_top_tracks):

In [6]:
# Se busca información sobre Blackpink y se obtiene su ID de artista.
result = sp.search(q='Blackpink', type='artist', limit=1)
blackpink_id = result['artists']['items'][0]['id']

# Se obtienen las 10 canciones más escuchadas de Blackpink.
results = sp.artist_top_tracks(blackpink_id)

# Se imprime el nombre de las canciones.
for track in results['tracks']:
    print(track['name'])

Pink Venom
Shut Down
How You Like That
Kill This Love
As If It's Your Last
Lovesick Girls
Kiss and Make Up
DDU-DU DDU-DU
Typa Girl
Pretty Savage


<br>

## **Extrayendo datos en tablas**

Creamos una función para la extracción de 50 canciones de un **género** definido (parámetro de entrada), sus características, y ordena la información en un dataFrame de **Pandas**:

In [22]:
def get_tracks_by_genre(genre):
    """
    Función que obtiene información de 50 canciones de un género musical de
    Spotify y organiza la información de un Pandas dataframe.
    """

    # Se buscan 50 canciones del género especificado.
    results = sp.search(q=f'genre:{genre}', type='track', limit=50)

    # Se extrane los IDs de las canciones.
    track_ids = [track['id'] for track in results['tracks']['items']]

    # Se obtienen detalles de las canciones.
    tracks_info = sp.tracks(tracks=track_ids)

    # Se crea una lista para almacenar los datos.
    track_data = []

    # Se obtienen los datos de cada canción y se añaden a la lista.
    for track in tracks_info['tracks']:
        track_data.append({
            'genre': genre,
            'name': track['name'],
            'artists_name': ', '.join([artist['name'] for artist in track['artists']]),  # Si hay varios artistas
            'album_name': track['album']['name'],
            'album_release_date': track['album']['release_date'],
            'duration_ms': track['duration_ms'],
            'popularity': track['popularity'],
            'track_number': track['track_number'],
            'uri': track['uri']
        })

    # Se convierte la lista en un DataFrame de pandas.
    df = pd.DataFrame(track_data)

    return df

Probamos la función con el género **Rock**:

In [23]:
# Ejemplo de uso con el género 'rock'.
genre = "rock"
get_tracks_by_genre(genre)

,genre,name,album_name,album_release_date,artists_name,duration_ms,popularity,track_number,uri
0,rock,Sweater Weather,I Love You.,2013-04-22,The Neighbourhood,240400,89,4,spotify:track:2QjOHCTQ1Jl3zawyYOpxh6
1,rock,505,Favourite Worst Nightmare (Standard Version),2007-04-24,Arctic Monkeys,253586,79,12,spotify:track:58ge6dfP91o9oXMzq3XkIS
2,rock,Rockin' Around The Christmas Tree,Merry Christmas From Brenda Lee,1964-10-19,Brenda Lee,126266,86,1,spotify:track:2EjXfH91m7f8HiJN1yQg97
3,rock,Mr. Brightside,Hot Fuss,2004,The Killers,222973,86,2,spotify:track:003vvx7Niy0yvhvHt4a68B
4,rock,Dreams - 2004 Remaster,Rumours (Super Deluxe),1977-02-04,Fleetwood Mac,257800,85,2,spotify:track:0ofHAoxe9vBkTCp2UQIavz
5,rock,Take Me to Church,Hozier (Expanded Edition),2014-09-19,Hozier,241693,79,1,spotify:track:1CS7Sd1u5tWkstBhpssyjP
6,rock,I Wanna Be Yours,AM,2013-09-09,Arctic Monkeys,183956,90,12,spotify:track:5XeFesFbtLpXzIVDNQP22n
7,rock,Cigarette Daydreams,Melophobia,2013-10-08,Cage The Elephant,208760,79,10,spotify:track:2tznHmp70DxMyr2XhWLOW0
8,rock,Last Christmas,LAST CHRISTMAS,1984-11-29,Wham!,262960,86,1,spotify:track:2FRnf9qhLbvw8fu4IBXx78
9,rock,Creep,Pablo Honey,1993-02-22,Radiohead,238640,86,2,spotify:track:70LcF31zb1H0PyJoS1Sx1r


**Nota**: una lista de los géneros disponibles se puede consultar en https://everynoise.com/everynoise1d-reggaetoncristiano.html

Ahora creamos un bucle que obtenga estos datos para cada género especificado de una **lista**, y concatenamos las **tablas** resultantes para obtener un conjunto de datos:

In [18]:
# Se importa una librería para la generación de una barra de progreso.
from tqdm import tqdm

# Se crea una lista con los géneros musicales deseados.
genres_list = ['world-music', 'salsa', 'rock-n-roll', 'reggae', 'reggaeton', 'pop', 'black-metal', 'k-pop', 'hip-hop', 'electronic']

# Se itera sobre esta lista utilizando sus elementos como entradas de la función creada.
tables = []

for item in tqdm(genres_list, desc = 'Creando conjunto de datos:'):
  table = get_tracks_by_genre(item)
  tables.append(table)

# Se concatenan las tablas y se imprime el resultado.
dataset = pd.concat(tables)
dataset

Creando conjunto de datos:: 100%|██████████| 10/10 [00:05<00:00,  1.84it/s]


,genre,name,album/name,album/release_date,artists/name,duration_ms,popularity,track_number,uri
0,world-music,Oceans (Where Feet May Fail),Zion (Deluxe Edition),2013-02-26,"Hillsong UNITED, TAYA",535962,66,4,spotify:track:5Mw9bXG1dLNhbjofkVS2oR
1,world-music,God Only Knows,Burn The Ships,2018-10-05,for KING & COUNTRY,229760,66,3,spotify:track:4ElNxglBjcrASiGn58t9Jm
2,world-music,So Will I (100 Billion X),Wonder,2017-06-09,"Hillsong UNITED, Benjamin William Hastings",411000,62,4,spotify:track:1ed3wNCecRIp6SugyR0w2J
3,world-music,For God Is With Us,What Are We Waiting For?,2022-03-11,for KING & COUNTRY,196846,63,5,spotify:track:0CQPPHjPpG4kAzvfOcKvh8
4,world-music,Lead Me To The Cross,All Of The Above,2007,"Hillsong UNITED, Brooke Ligertwood",258626,58,6,spotify:track:7i2ZmmyLzE7NcFAJl30mKY
...,...,...,...,...,...,...,...,...,...
45,electronic,Tsunami,Tsunami,2013-09-02,"DVBBS, Borgeous",236706,65,1,spotify:track:0vZCG0H9KhtU7K8MEUVAoV
46,electronic,Steal,Portraits,2015-06-01,"Maribou State, Holly Walker",219945,67,4,spotify:track:0pAiyIHt9DyHOjWgF41kp6
47,electronic,Bfg Division,Doom (Original Game Soundtrack),2016-09-28,Mick Gordon,506762,60,11,spotify:track:4COR2ZPEyUn0lsbAouRWxA
48,electronic,Six Days - Remix,The Fast And The Furious: Tokyo Drift (Origina...,2006-06-20,"DJ Shadow, Mos Def",232000,67,2,spotify:track:5j3QqRGflS4o5jbsFSwKW1


Por último se descarga en formato CSV:

In [19]:
# Se convierte a formato CSV.
dataset.to_csv('dataset.csv', index=False)

# Se descarga el CSV.
from google.colab import files
files.download('dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

O en formato XLS:

In [ ]:
# Se convierte a formato Excel.
dataset.to_excel('dataset.xlsx', index=False)

# Se descarga el Excel.
from google.colab import files
files.download('dataset.xlsx')

No olvides que puedes encontrar más tópicos como este en https://dialektico.com/.

▶ [Regresar al tutorial](https://dialektico.com/tutorial-api-spotify-python/) 📕

<br>

In [ ]:
# Dialektico Machine learning practices © 2024 by Daniel Antonio García Escobar
# is licensed under CC BY-NC 4.0. To view a copy of this license,
# visit https://creativecommons.org/licenses/by-nc/4.0/

# Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International
# Public License